**IMPORTANT** 

- For requirements and initial setup go to https://github.com/OliveiraEdu/OpenScience/Readme.md;
- To execute the notebook run all cells.

# Open Science Platform Notebook 1 -User Account Creation

## Activities

1 - Deploys a smart contract into the Iroha 1 blockchain for account creation;

2 - Creates a user account into the blockchain;

3 - Queries the blockchain to confirm the succesful creation of the account;

4 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

5 - Sets details for the user account;

6 - Queries the blockchain again to confirm the succesfull details setting.

User accounts and their respective attributes are randomly picked, every account succesfully created is dumped to `datasets/accounts.json` for later use and evidence collection.  

In [1]:
from name_generator import left, right
from utilities import *
from dump_to_json import *
from iroha_helper import *
from new_helper import extract_account_metadata_cid_from_data, process_raw_data, load_variable
from loguru import logger

In [2]:
DOMAIN = "test"

1 - Loads the smart contract hash for account creation into the Iroha 1 blockchain.

In [3]:
# Loading the variable from JSON
create_account_contract_hash = load_variable("Create Account Hash")
logger.info(create_account_contract_hash)


2025-01-26 22:19:24,211 - INFO - Variable 'Create Account Hash' successfully loaded from temp/Create Account Hash.json
2025-01-26 22:19:24.213 | INFO     | __main__:<module>:3 - b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'


2 - Creates a user account into the blockchain.

In [4]:
user_private_key = IrohaCrypto.private_key()
user_public_key = IrohaCrypto.derive_public_key(user_private_key).decode("utf-8")

# - Random generated synthetic user data 
user_account_left = random.choice(left)
user_account_right = random.choice(right)
user_account_short_id = f"{user_account_left}_{user_account_right}"
user_account_full_name = ((f"{user_account_left}").capitalize())+" "+((f"{user_account_right}").capitalize())
user_account_email = f"{user_account_left}_{user_account_right}"+"@email.com"
user_account_institution = print_random_from_second_column("datasets/world-universities.csv")
user_account_orcid = generate_orcid()
user_role = set_random_role()
logger.info(f"User Role: {user_role}")

user_account = UserAccount(
    account_id=f"{user_account_short_id}@{DOMAIN}",
    full_name=user_account_full_name,
    email=user_account_email,
    institution=user_account_institution,
    orcid=user_account_orcid,
    role=user_role,
    public_key=user_public_key,
)
logger.info(f"Creating account with name: {user_account_short_id}")

# - Create account
create_account_contract_address = integration_helpers.get_engine_receipts_address(create_account_contract_hash)
create_user_account(create_account_contract_address, user_account_short_id, DOMAIN, user_public_key, user_account)

2025-01-26 22:19:24.268 | INFO     | __main__:<module>:13 - User Role: publisher
2025-01-26 22:19:24.270 | INFO     | __main__:<module>:24 - Creating account with name: great_black
2025-01-26 22:19:24.272 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 22:19:24.294 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'3981e30acae9fd80ff03e6b1f0338631e32f43c8ca9c0a612fdfd4c458a9b8d9'
2025-01-26 22:19:24.296 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 22:19:24.298 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 22:19:24.300 | DEBUG    | integration_helpers:tracer:35 - 	Entering "create_user_account"


2025-01-26 22:19:24,320 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 22:19:24,322 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 22:19:26,190 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 22:19:26,203 - INFO - ('COMMITTED', 5, 0)
2025-01-26 22:19:26.207 | INFO     | dump_to_json:dump_to_json_ld:82 - Appended new entry to file 'datasets/accounts.json', current total entries: 1
2025-01-26 22:19:26.208 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "create_user_account"


b'61c8d3a73ee1288db2bab69e6c4cd981a5e850a61a03dd25a2ea74db7b26469f'

3 - Queries the blockchain to confirm the succesful creation of the account.

In [5]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
address = integration_helpers.get_engine_receipts_result(get_account_hash)

2025-01-26 22:19:26.232 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 22:19:26,234 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 22:19:26,248 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 22:19:26,249 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 22:19:29,230 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 22:19:29,238 - INFO - ('COMMITTED', 5, 0)
2025-01-26 22:19:29,240 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': '01ae0145dacf6f06ecf608ff67cb277c1c452bc79fe3dee656e1c244ea1f7ccf'}
2025-01-26 22:19:29.241 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 22:19:29,245 - INFO - Appended data under 'great_black@test' in logs/account_data.json
2025-01-26 22:19:29.246 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 22:19:29.247 | DEBUG    | integrat

4 - Loads the smart contract for details (attributes) setting into the Iroha 1 blockchain .

In [6]:
# Loading the variable from JSON
account_detail_contract_hash = load_variable("Account Detail Contract Hash")
logger.info(account_detail_contract_hash)

2025-01-26 22:19:29,280 - INFO - Variable 'Account Detail Contract Hash' successfully loaded from temp/Account Detail Contract Hash.json
2025-01-26 22:19:29.284 | INFO     | __main__:<module>:3 - b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'


5 - Sets details for the user account:

- Full Name
- Email
- Institution
- ORCID
- Public key
- Role

In [7]:
address = integration_helpers.get_engine_receipts_address(account_detail_contract_hash)

process_account(address, f"{user_account_short_id}@{DOMAIN}")

2025-01-26 22:19:29.308 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_engine_receipts_address"
2025-01-26 22:19:29.325 | INFO     | integration_helpers:get_engine_receipts_address:96 - Retrieved contract address for transaction hash: b'64b442bbecf02230ecf460f75eadcbbcce3334b831185b81d99e2f3f159a962d'
2025-01-26 22:19:29.326 | DEBUG    | integration_helpers:get_engine_receipts_address:97 - Contract address: ECAB4A3243301E6F3C6EABAC6D6DE1CE7E703118
2025-01-26 22:19:29.327 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "get_engine_receipts_address"
2025-01-26 22:19:29.329 | DEBUG    | integration_helpers:tracer:35 - 	Entering "process_account"
2025-01-26 22:19:29,330 - INFO - Processing account: great_black@test
2025-01-26 22:19:29,332 - INFO - User Account Metadata: {'@type': 'foaf:Person', 'foaf:name': 'Great Black', 'foaf:mbox': 'great_black@email.com', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Effat College'}, 'schema:identifier': {'@type':

6 - Queries the blockchain again to confirm the succesfull details setting.

In [8]:
# - Get account info
get_account_hash = get_account(create_account_contract_address, user_account_short_id, DOMAIN)
details = integration_helpers.get_engine_receipts_result(get_account_hash)

# Process the raw data
account_metadata_cid = process_raw_data(details)

2025-01-26 22:19:32.337 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_account"
2025-01-26 22:19:32,339 - WARNING - create_account_contract_address: FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70
2025-01-26 22:19:32,357 - INFO - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-26 22:19:32,360 - INFO - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-26 22:19:35,320 - INFO - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-26 22:19:35,328 - INFO - ('COMMITTED', 5, 0)
2025-01-26 22:19:35,330 - WARNING - Account Data: {'address': 'FF762AC4E3D64AC77F560CC0DB6D0623F32E8F70', 'hex_hash': '7b785ac53edf59c82852f4055847bf39d310e20a8511c6ae6a333ffce61382a0'}
2025-01-26 22:19:35.331 | DEBUG    | integration_helpers:tracer:35 - 	Entering "append_to_json_file"
2025-01-26 22:19:35,336 - INFO - Appended data under 'great_black@test' in logs/account_data.json
2025-01-26 22:19:35.337 | DEBUG    | integration_helpers:tracer:37 - 	Leaving "append_to_json_file"
2025-01-26 22:19:35.339 | DEBUG    | integrat

7 - Retrieves User Metadata (JSON-LD formatted) from IPFS

In [9]:
user_metadata = download_json_from_ipfs(account_metadata_cid)
logger.info(f"User Metadata: {user_metadata}")

2025-01-26 22:19:35.402 | INFO     | __main__:<module>:2 - User Metadata: {'@type': 'foaf:Person', 'foaf:holdsAccount': {'schema:identifier': 'great_black@test', 'schema:publicKey': 'cf698d65a3f8b17d684e64de9dda1c0e7c220b53079e5680bd10a8c70c70bcbb', 'schema:roleName': 'publisher'}, 'foaf:mbox': 'great_black@email.com', 'foaf:name': 'Great Black', 'foaf:organization': {'@type': 'foaf:Organization', 'foaf:name': 'Effat College'}, 'schema:identifier': {'@type': 'PropertyValue', 'propertyID': 'ORCID', 'value': '2210-3293-8398-X'}}
